### Explanation
This code analyse data (i.e, position,growth and fluorescence) from time lapse images of plates with growing colonies expressing different fluorescent proteins (colors)

##### Requirements
 - Image Data


## Analysis


### Loading packages

In [ ]:
# import some useful packages to manage the data

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#modify some matplotlib parameters to manage the images for illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
# Import the FluoPi module
import fluopi.analysis as flua
import fluopi.plotting as flup

### Loading Data

*** it could be reemplaced by loading the data obtained qith the code "Colony size and fluorescence' (or have the option to do that)

In [ ]:
fname1 =  "D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_G_7_3_17\image_%04d.jpg"
#refered to drive folder TLE_G_7_3_17

fname2 = "D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_O_10_3_17\image_%04d.jpg"
#refered to drive folder TLE_O_10_3_17

fname3 = "D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_R_13_3_17\image_%04d.jpg"
#refered to drive folder TLE_R_13_3_17

fname4 = "D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_RGO_17_3_17\image_%04d.jpg"
#refered to drive folder TLE_RGO_17_3_17

In [ ]:
#let's try this with the frame 500 image 
flup.plot_im_frame(fname2,200)

In [ ]:
flup.plt_im_frame_channels(fname2,200)

In [ ]:
imCount1=flua.count_files('D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_G_7_3_17',"jpg")
imCount2=flua.count_files('D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_O_10_3_17',"jpg")
imCount3=flua.count_files('D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_R_13_3_17',"jpg")
imCount4=flua.count_files('D:\Dropbox\FTI\Raspiscope\Time lapse\TLE_RGO_17_3_17',"jpg")

In [ ]:
DataS1={}
frames1 = 10   # use one image every 10 frames
DataS1['R'],DataS1['G'],DataS1['B']=flua.get_im_data(frames1,imCount1,fname1)
DataS1['Im']=fname1     # to store the related image source

In [ ]:
DataS2={}
frames2 = 5
DataS2['R'],DataS2['G'],DataS2['B']=flua.get_im_data(frames2,imCount2,fname2)
DataS2['Im']=fname2

In [ ]:
DataS3={}
frames3 = 5
DataS3['R'],DataS3['G'],DataS3['B']=flua.get_im_data(frames3,imCount3,fname3) 
DataS3['Im']=fname3

In [ ]:
DataS4={}
frames4 = 5
DataS4['R'],DataS4['G'],DataS4['B']=flua.get_im_data(frames4,imCount4,fname4)
DataS4['Im']=fname4

Now we have each data serie on a dictionary, organized in arrays for each channel.

### Background subtraction
First step is to identify a good background area and compute his value over time:

In [ ]:
BG1=flua.bg_value(200,370,10,400,DataS1,imCount1)

In [ ]:
BG2=flua.bg_value(180,360,420,750,DataS2,imCount2)

In [ ]:
BG3=flua.bg_value(100,380,50,430,DataS3,imCount3)

In [ ]:
BG4=flua.bg_value(450,670,260,420,DataS4,imCount4)

we can substract that vector value to each frame on the image to eliminate the background effect

In [ ]:
DataSB1=flua.bg_subst(DataS1,BG1)

In [ ]:
DataSB2=flua.bg_subst(DataS2,BG2)

In [ ]:
DataSB3=flua.bg_subst(DataS3,BG3)

In [ ]:
DataSB4=flua.bg_subst(DataS4,BG4)

### Get colony position
a good way to identify where colonies are situated is by sum the value of each pixel for all the channels and all the frames

In [ ]:
SumData1=flua.data_sum_time(DataSB1)

In [ ]:
SumData2=flua.data_sum_time(DataSB2)

In [ ]:
SumData3=flua.data_sum_time(DataSB3)

In [ ]:
SumData4=flua.data_sum_time(DataSB4)

####  apply a filter to smooth the data (reduce noise and improve border detection)

In [ ]:
sDatS1,sDatSall1,SDatST1=flua.smooth_data(DataSB1,1.5)

In [ ]:
sDatS2,sDatSall2,SDatST2=flua.smooth_data(DataSB2,1.5)

In [ ]:
sDatS3,sDatSall3,SDatST3=flua.smooth_data(DataSB3,1.5)

In [ ]:
sDatS4,sDatSall4,SDatST4=flua.smooth_data(DataSB4,1.5)

### Blob detection

In [ ]:
blobD1 = flua.colony_blobs_id(sDatS1['G'],0.35,DataS1['Im']%(imCount1-1)) 
#for green it was better to use only the green channel

#blobD1 = flua.colony_blobs_id(sDatS1['G'],0.35,DataS1['Im']%(imCount1-1),filename='Plots/Blobs_sfGFP') 

In [ ]:
blobD2 = flua.colony_blobs_id(sDatSall2,0.34,DataS2['Im']%(imCount2-1))
#blobD2 = flua.colony_blobs_id(sDatSall2,0.34,DataS2['Im']%(imCount2-1),filename='Plots/Blobs_CyOFP')

In [ ]:
blobD3 = flua.colony_blobs_id(sDatSall3,0.34,DataS3['Im']%(imCount3-1))
#blobD3 = fluopi.flua.colony_blobs_id(sDatSall3,0.34,DataS3['Im']%(imCount3-1),filename='Plots/Blobs_BeRFP')

In [ ]:
blobD4 = flua.colony_blobs_id(sDatSall4,0.37,DataS4['Im']%(imCount4-1))
#blobD4 = flua.colony_blobs_id(sDatSall4,0.37,DataS4['Im']%(imCount4-1),filename='Plots/Blobs_MixFPs')

In [ ]:
Rois1,RoisC1,NC1 = flua.obtain_rois(DataSB1,blobD1)
Rois2,RoisC2,NC2 = flua.obtain_rois(DataSB2,blobD2)
Rois3,RoisC3,NC3 = flua.obtain_rois(DataSB3,blobD3)
Rois4,RoisC4,NC4 = flua.obtain_rois(DataSB4,blobD4)

## ROIS Color Analysis

In [ ]:
ChanX='G'
ChanY='R'
Xchan1,Ychan1=flup.rois_last_frame_2chan_plt(RoisC1,ChanX,ChanY,'Serie sfGFP')

In [ ]:
Xchan2,Ychan2=flup.rois_last_frame_2chan_plt(RoisC2,ChanX,ChanY,'Serie BeRFP')

In [ ]:
Xchan3,Ychan3=flup.rois_last_frame_2chan_plt(RoisC3,ChanX,ChanY,'Serie CyOFP')

In [ ]:
Xchan4,Ychan4=flup.rois_last_frame_2chan_plt(RoisC4,ChanX,ChanY,'Serie Mix')

In [ ]:
fit1=flua.linear_fit(Xchan1[:,0],Ychan1[:,0])
#fit1=flua.linear_fit(Xchan1[:,0],Ychan1[:,0],filename="Plots/RG_fit1.pdf")

In [ ]:
fit2=flua.linear_fit(Xchan2[:,0],Ychan2[:,0])
#fit2=flua.linear_fit(Xchan2[:,0],Ychan2[:,0],filename="Plots/RG_fit2.pdf")

In [ ]:
fit3=flua.linear_fit(Xchan3[:,0],Ychan3[:,0])
#fit3=linear_fit(Xchan3[:,0],Ychan3[:,0],filename="Plots/RG_fit3.pdf")

In [ ]:
ClasS4,NClasS4=flua.colony_classifier([fit1,fit2,fit3],['GFP','CyOFP','BeRFP'],Xchan4,Ychan4)

In [ ]:
Xmin=0
flup.plt_lin_fit(Xmin,90000,fit1,'g')
flup.plt_lin_fit(Xmin,90000,fit2,'y')
flup.plt_lin_fit(Xmin,30000,fit3,'r')
plt.plot(NClasS4['GFP'][0],NClasS4['GFP'][1],'go')
plt.plot(NClasS4['CyOFP'][0],NClasS4['CyOFP'][1],'yo')
plt.plot(NClasS4['BeRFP'][0],NClasS4['BeRFP'][1],'ro')
#plt.savefig("Plots/Classification.pdf", transparent=True)